In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Dense, Lambda
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
#     vgg_out = vgg_model.layers[-2].output #Last FC layer's output 
    layer_name = 'fc2'
    x = vgg_model.get_layer(layer_name).output
    temper = 1.0;
    x = Lambda(lambda x: x/temper)(x)
    
    layer_name = 'predictions'
    pred_layer = vgg_model.get_layer(layer_name) 
    weight = pred_layer.get_weights()
    x = Dense(1000, activation='softmax', name='softmax1', weights=weight)(x) 
    
    softmax_layer = Dense(256, activation='softmax')(x) #Create softmax layer taking input as vgg_ou
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False
        
    layer_name = 'softmax1'
    tl_model.get_layer(layer_name).trainable = False
#     tl_model.get_layer(layer_name).set_weights(weights)
    
    tl_model.summary()

    return tl_model
def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 16
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [3]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [6]:
#Output dim for your dataset
output_dim = 256 #For Caltech256

#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [11]:
#Model
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error

# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/Keras-1.2.2-py2.7.egg/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 4096)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [12]:
# sgd = keras.optimizers.SGD(lr=16, momentum=0.9, decay=0.0, nesterov=True)
# tl_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

adagrad = keras.optimizers.Adagrad(lr=1, epsilon=1e-08, decay=0.0)
tl_model.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['accuracy'])

In [13]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [14]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 2048 samples, validate on 2048 samples
Epoch 1/20
2048/2048 [==============================] - 82s - loss: 4.2969 - acc: 0.2305 - val_loss: 3.2597 - val_acc: 0.3623
Epoch 2/20
2048/2048 [==============================] - 82s - loss: 1.9735 - acc: 0.6104 - val_loss: 3.0133 - val_acc: 0.4097
Epoch 3/20
2048/2048 [==============================] - 83s - loss: 1.3802 - acc: 0.7285 - val_loss: 2.9885 - val_acc: 0.4229
Epoch 4/20
2048/2048 [==============================] - 83s - loss: 1.0656 - acc: 0.8071 - val_loss: 3.0122 - val_acc: 0.4229
Epoch 5/20
2048/2048 [==============================] - 72s - loss: 0.8670 - acc: 0.8457 - val_loss: 3.0498 - val_acc: 0.4277
Epoch 6/20
2048/2048 [==============================] - 41s - loss: 0.7310 - acc: 0.8765 - val_loss: 3.0911 - val_acc: 0.4312
Epoch 7/20
2048/2048 [==============================] - 41s - loss: 0.6331 - acc: 0.8979 - val_loss: 3.1323 - val_acc: 0.4263
Epoch 8/20
2048/2048 [==============================] - 41s - loss: 0.

In [15]:
print(hist.history)

{'acc': [0.23046875, 0.6103515625, 0.728515625, 0.80712890625, 0.845703125, 0.87646484375, 0.89794921875, 0.9169921875, 0.92822265625, 0.93505859375, 0.9443359375, 0.9521484375, 0.95703125, 0.9599609375, 0.962890625, 0.9638671875, 0.9658203125, 0.96826171875, 0.9697265625, 0.970703125], 'loss': [4.2969001568853855, 1.973468229174614, 1.3801582744345069, 1.06562574300915, 0.86696815490722656, 0.73100262088701129, 0.63307825848460197, 0.55979155888780951, 0.50319365644827485, 0.45831487374380231, 0.42192403180524707, 0.3918494344688952, 0.36658694734796882, 0.34506615134887397, 0.32650665054097772, 0.31032778369262815, 0.29608941031619906, 0.28345258231274784, 0.27215242153033614, 0.26197918271645904], 'val_acc': [0.3623046875, 0.40966796875, 0.4228515625, 0.4228515625, 0.427734375, 0.43115234375, 0.42626953125, 0.42431640625, 0.4228515625, 0.42236328125, 0.42236328125, 0.42138671875, 0.42138671875, 0.42041015625, 0.4208984375, 0.42041015625, 0.4208984375, 0.42041015625, 0.41943359375, 0